In [64]:
import requests
import pandas as pd
import csv
import re

In [68]:
date_range = ["2012-01-01", "2021-12-31"]
api_url = "https://mapacultural.secult.ce.gov.br/api"

occurrences_path = "/event/findByLocation"
occurrences_params = {
    '@from': date_range[0],
    '@to': date_range[1],
    '@select': 'id,name,classificacaoEtaria,occurrences.{*}'
}
headers = {
    'User-Agent': 'Mozilla/5.0'
}
def replace_price(s):
    found = re.search(r'[1-9](\d+)?', s)
    num = int(found.group(0)) if found else 0
    return num if num < 5000 else 0

print("Getting occurences from %s to %s, this may take a while." % (date_range[0], date_range[1]))

resp = requests.get(url=api_url+occurrences_path, params=occurrences_params, headers=headers)
events = resp.json()
df = pd.DataFrame()

# print(events)

# Loop over occurrences (with tqdm for progress)
print("Getting events for each occurrence.")
for event in events:
    data = event["occurrences"][0]["rule"]
    event_meta = {
        "price": replace_price(data["price"]),
        "duration": int(data["duration"]) / 60,
        "frequency": data["frequency"]
    }
    df = pd.concat([df, pd.json_normalize(event_meta)])

df.to_csv('mapa.csv', index=None)

df

Getting occurences from 2012-01-01 to 2021-12-31, this may take a while.
Getting events for each occurrence.


,price,duration,frequency
0,0,2.0,once
0,0,4.0,once
0,10,3.0,daily
0,0,3.0,once
0,2,19.0,once
...,...,...,...
0,0,2.0,once
0,0,2.0,once
0,0,3.0,once
0,0,2.0,once
